In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
scales = [3]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = torch.logical_or(x<30, x>50)
    filtered_x = s.where(filter, x)
    return s.pstdev(filtered_x)

error = 0.01
_, prover_model = computation_to_model(computation, precal_witness_path, True, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(True)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is depre

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 3, param_scale: 3, scale_input_multiplier: 1) ------------->

+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.028796673 | 0.057593346  | 0.057593346 | 0         | 0.028796673    | 0.057593346      | 0.057593346   | 0             | 0.0016584967       | 0.0018074065       | 0.0018074065           |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+----

scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":1,"lookup_range":[-13376,13448],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":14757,"total_const_size":5130,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-4,4]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715153603898}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False,error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 3.6159000396728516 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 15.875
==== Generating Proof ====
proof:  {'instances': [['d57f47950cdabf2cb79306e0f33e75726a2c2960806e902b0fc88d3ff949a108', '0100000000000000000000000000000000000000000000000000000000000000', '7f00000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2b595bf866a9d8bc00d8713e971f8d66f60d3d5c176a20191e9475b73c7733bb05085e38138c5a13ff74f9852f4387ce90822d4c0ea2b7da20c778d4b92da63625e0ca6f1a7f7ce3471cacdde7df506c49a145e3508c29acc4ed6517bf4263ce1491a72c98dd1cd16d120b0425032d64ff0f4bcd60f1289734f10584cbee4c361133aa0b22cac37141cb10c9a98ed3cc4c4467066dc3f558583822fb12c67a3c062d461e4c70ee5e66e8b903836e0ba2745324f9b627ada7004c93cb6292d3f82f4840ef378176fe624c2b9c88216bdcc24ef8450bc10980f84f20f3294a75c02b0a572bc3f20bfda21d723590ef6b5546beaef6e2290b2416ec6b8ae504e7ed11325a47f69d1333e802334eae6f31b1b72bcf072128637d32ebdd72ea0b186a136e45b604900

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [15.875]
